In [3]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme

In [4]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [5]:
# !bash report.sh
organisms = set(pandas.read_csv("biomass_constrained.txt",index_col=0,header=None).index.to_list())
done = set(i.split(".csv")[0] for i in os.listdir("./cases/fluxes/nonIBD/") if "csv" in i)

In [6]:
len(organisms)

495

### Load

In [21]:
def load_data(path):
    try:
        df = pandas.read_csv(path,index_col=0)[["fluxes"]]
    except:
        df = pandas.DataFrame(columns=["fluxes"])
    df.columns = [org]
    return df

In [22]:
from tqdm import tqdm
conditions = ["nonIBD", "IBD"]

In [30]:
overflow = {i:pandas.DataFrame() for i in conditions}
growth = {i:{} for i in conditions}

In [24]:
for org in tqdm(organisms):
    for c in conditions:
        tmp = load_data("./cases/overflow/{}/{}.csv".format(c,org))
        overflow[c] = pandas.concat([overflow[c],tmp], axis=1).fillna(0)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [00:01<00:00, 297.83it/s]


In [25]:
for org in tqdm(organisms):
    for c in conditions:
        tmp = load_data("./cases/fluxes/{}/{}.csv".format(c,org))
        if tmp.empty:
            val = None
        else:
            val = tmp[org]["biomass_dilution"]
        growth[c][org] = val
growth = pandas.DataFrame.from_dict(growth)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [00:26<00:00, 18.95it/s]


In [26]:
growth.dropna()

,nonIBD,IBD
Capnocytophaga_sputigena_ATCC_33612,0.003977,0.006603
Brachyspira_pilosicoli_B2904,0.001114,0.001134
Listeria_monocytogenes_SLCC_2378,0.001697,0.001697
Providencia_rustigianii_DSM_4541,0.001154,0.001298
Bacteroides_xylanisolvens_SD_CC_1b,0.012298,0.130969
...,...,...
Lactobacillus_helveticus_DPC_4571,0.001009,0.001070
Lactobacillus_delbrueckii_subsp_bulgaricus_ATCC_BAA_365,0.001341,0.001341
Bacteroides_sp_2_1_7,0.093961,0.102222
Edwardsiella_tarda_FL6_60,0.001102,0.001349


### Save

In [27]:
growth.sort_index().to_csv("datasets/3.5.IBD_growth_rates.csv")

In [28]:
for c,df in overflow.items():
    df.to_csv("datasets/3.5.{}_overflow.csv".format(c))

In [7]:
growth = pandas.read_csv("datasets/3.5.IBD_growth_rates.csv",index_col=0)

,nonIBD,IBD
Abiotrophia_defectiva_ATCC_49176,0.001993,NaN
Acidaminococcus_fermentans_DSM_20731,0.001525,0.001937
Acidaminococcus_intestini_RyC_MR95,0.001026,0.002054
Acidaminococcus_sp_D21,0.002297,0.055847
Acinetobacter_calcoaceticus_PHEA_2,0.001129,0.000918
...,...,...
Yersinia_enterocolitica_subsp_enterocolitica_8081,0.002792,0.001391
Yersinia_frederiksenii_ATCC_33641,0.004153,0.003634
Yersinia_kristensenii_ATCC_33638,0.000870,0.001085
Yersinia_pseudotuberculosis_YPIII,0.007851,0.001409
